# Analysis of the presidential election

#### Author: Bartosz Dzionek ([Github](https://github.com/dzionek))

We will use the following libraries through this analysis:

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px

## Introduction

In this Data Science paper I analyze the results of the presidential elections in Poland in 2020. The results are taken from the website of the National Electoral Commission (Polish: *Państwowa Komisja Wyborcza*) – [link](https://wybory.gov.pl/prezydent20200628/en/dane_w_arkuszach).

In this election the voters were eligible to vote for the following candidates:

In [2]:
candidates_df = pd.read_csv('data/candidates/candidates.csv', sep=';')
candidates_df.set_index('Pozycja na karcie', inplace=True)

In [3]:
candidates_df

,Nazwisko,Imiona,Płeć,Wiek,Wykształcenie,Zawód,Miejsce pracy,Miejsce zamieszkania,Przynależność do partii,Nazwa komitetu,Sygnatura,TERYT m. z.,Gmina m.z.
Pozycja na karcie,,,,,,,,,,,,,
1,BIEDROŃ,Robert,Mężczyzna,44,wyższe politologiczne,poseł do Parlamentu Europejskiego,NaN,Warszawa,członek partii politycznej: Wiosna Roberta Bie...,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-6/20,146506,Ochota
2,BOSAK,Krzysztof,Mężczyzna,38,średnie,poseł na Sejm Rzeczypospolitej Polskiej,NaN,Warszawa,członek partii politycznej: Konfederacja Wolno...,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-1/20,146518,Wola
3,DUDA,Andrzej Sebastian,Mężczyzna,48,wyższe prawnicze,Prezydent Rzeczypospolitej Polskiej,NaN,Kraków,nie należy do partii politycznej,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-5/20,126101,m. Kraków
4,HOŁOWNIA,Szymon Franciszek,Mężczyzna,43,średnie,publicysta,własna działalność gospodarcza,Otwock,nie należy do partii politycznej,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-4/20,141702,m. Otwock
5,JAKUBIAK,Marek,Mężczyzna,61,średnie,menadżer,Browary Regionalne Jakubiak Sp. z o.o.,Warszawa,członek partii politycznej: Federacja dla Rzec...,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-28/20,146510,Śródmieście
6,KOSINIAK-KAMYSZ,Władysław Marcin,Mężczyzna,38,wyższe medyczne,poseł na Sejm Rzeczypospolitej Polskiej,NaN,Kraków,członek partii politycznej: Polskie Stronnictw...,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-2/20,126101,m. Kraków
7,PIOTROWSKI,Mirosław Mariusz,Mężczyzna,54,wyższe historyczne,nauczyciel akademicki,Wyższa Szkoła Kultury Społecznej i Medialnej,Lublin,członek partii politycznej: Ruch Prawdziwa Eur...,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-9/20,66301,m. Lublin
8,TANAJNO,Paweł Jan,Mężczyzna,44,wyższe w zakresie zarządzania,przedsiębiorca,IAM4U.pl Sp. z o.o.,Warszawa,nie należy do partii politycznej,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-13/20,146518,Wola
9,TRZASKOWSKI,Rafał Kazimierz,Mężczyzna,48,wyższe politologiczne,pracownik samorządowy,Urząd m.st. Warszawy,Warszawa,członek partii politycznej: Platforma Obywatel...,KOMITET WYBORCZY KANDYDATA NA PREZYDENTA RZECZ...,ZPOW-6020-41/20,146510,Śródmieście


We generate a list of these candidates to filer their results later on.

In [5]:
candidates_first_name = candidates_df['Imiona'].tolist()
candidates_last_name = candidates_df['Nazwisko'].tolist()

candidates = np.array(
    list(zip(candidates_first_name, candidates_last_name))
)

candidates = [candidate[0] + ' ' + candidate[1] for candidate in candidates]

In [6]:
candidates

['Robert BIEDROŃ',
 'Krzysztof BOSAK',
 'Andrzej Sebastian DUDA',
 'Szymon Franciszek HOŁOWNIA',
 'Marek JAKUBIAK',
 'Władysław Marcin KOSINIAK-KAMYSZ',
 'Mirosław Mariusz PIOTROWSKI',
 'Paweł Jan TANAJNO',
 'Rafał Kazimierz TRZASKOWSKI',
 'Waldemar Włodzimierz WITKOWSKI',
 'Stanisław Józef ŻÓŁTEK']

## Results

### Total results

Firstly, we find how many voters have voted for each candidate.

In [45]:
results_counties_df = pd.read_csv('data/results/results_by_county.csv', sep=';')

results_df = (
    results_counties_df[candidates].sum().to_frame('Result')
                                   .sort_values('Result', ascending=False)
                                   .reset_index()
)

results_df.columns = ['Candidate', 'Result']

In [46]:
results_df

,Candidate,Result
0,Andrzej Sebastian DUDA,8412183
1,Rafał Kazimierz TRZASKOWSKI,5845164
2,Szymon Franciszek HOŁOWNIA,2667655
3,Krzysztof BOSAK,1300923
4,Władysław Marcin KOSINIAK-KAMYSZ,457092
5,Robert BIEDROŃ,425734
6,Stanisław Józef ŻÓŁTEK,45092
7,Marek JAKUBIAK,33294
8,Paweł Jan TANAJNO,27610
9,Waldemar Włodzimierz WITKOWSKI,26877


In [53]:
fig = px.bar(results_df, x='Candidate', y='Result', title='Total number of votes')
fig.show()

But we also want to see the results in percents. We find the total number of votes to get that information.

In [55]:
total_num_of_votes = results_df['Result'].sum()
total_num_of_votes

19262568

In [59]:
def find_percent(n):
    """Find the percent of n with regards to the total number of votes."""
    return round(n / total_num_of_votes * 100, 2)

In [64]:
results_percent_df = pd.concat([results_df['Candidate'], results_df['Result'].map(find_percent)], axis=1)
results_percent_df

,Candidate,Result
0,Andrzej Sebastian DUDA,43.67
1,Rafał Kazimierz TRZASKOWSKI,30.34
2,Szymon Franciszek HOŁOWNIA,13.85
3,Krzysztof BOSAK,6.75
4,Władysław Marcin KOSINIAK-KAMYSZ,2.37
5,Robert BIEDROŃ,2.21
6,Stanisław Józef ŻÓŁTEK,0.23
7,Marek JAKUBIAK,0.17
8,Paweł Jan TANAJNO,0.14
9,Waldemar Włodzimierz WITKOWSKI,0.14


TODO: plot counties map

In [19]:
import geopandas as gpd

In [24]:
gdf = gpd.GeoDataFrame.from_file(
    'data/counties/counties.shp', encoding = 'utf-8'
)

In [25]:
gdf = gdf[['JPT_NAZWA_', 'JPT_KOD_JE', 'geometry']]

In [26]:
gdf.columns = ['county name', 'TERYT', 'geometry']

In [27]:
gdf

,county name,TERYT,geometry
0,powiat ropczycko-sędziszowski,1815,"POLYGON ((21.69429 50.17849, 21.69440 50.17849..."
1,powiat łosicki,1410,"POLYGON ((22.72381 52.28039, 22.72379 52.28043..."
2,powiat piaseczyński,1418,"POLYGON ((21.11010 52.11317, 21.10875 52.11459..."
3,powiat radomski,1425,"POLYGON ((20.98915 51.48947, 20.98919 51.48942..."
4,powiat sierpecki,1427,"POLYGON ((19.50876 52.86784, 19.50873 52.86795..."
...,...,...,...
375,powiat Żory,2479,"MULTIPOLYGON (((18.64184 50.06883, 18.64168 50..."
376,powiat nowodworski,2210,"POLYGON ((19.33018 54.36649, 19.33137 54.36671..."
377,powiat jaworski,0205,"POLYGON ((16.19848 51.01198, 16.19854 51.01167..."
378,powiat poznański,3021,"POLYGON ((16.60048 52.41223, 16.60044 52.41277..."
